In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125095")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R2LXSZ63K to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-125095
Azure region: southcentralus
Subscription id: de47103e-2da6-4f5e-88fc-d18b27fd249b
Resource group: aml-quickstarts-125095


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Nane of the CPU cluster
amlcompute_cluster_name = "my-compute"

# Verify thet cluster does not exist
try:
    aml_compute = ComputeTarget(workspace= ws, name=amlcompute_cluster_name)
    print('Found existing cluster, will use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(wm_size = 'STANDARD_D2_V2', max_nodes=4)
    aml_compute.wait_for_completion(show_output=True)

Found existing cluster, will use it!


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({
    '--C': uniform(0.5, 1.5),
    '--max_iter': choice(50, 150)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.085)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target='my-compute',
entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=10,
max_concurrent_runs=2)
print('HyperDriveConfig has been created!')

HyperDriveConfig has been created!


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

my_run = exp.submit(config=hyperdrive_config)
RunDetails(my_run).show()
my_run.wait_for_completion(show_output=True)
assert(my_run.get_status()=="Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0bca8978-eddd-4fce-8d8a-fff6fca29fb7
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_0bca8978-eddd-4fce-8d8a-fff6fca29fb7?wsid=/subscriptions/de47103e-2da6-4f5e-88fc-d18b27fd249b/resourcegroups/aml-quickstarts-125095/workspaces/quick-starts-ws-125095

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-07T20:39:44.859109][API][INFO]Experiment created<END>\n"<START>[2020-11-07T20:39:45.5941195Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-07T20:39:45.663492][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-11-07T20:39:46.093935][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_0bca8978-eddd-4fce-8d8a-fff6fca29fb7
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_0bca8978-eddd-4fce-8d8a-fff6fca29fb7?wsid=/subscriptions/de471

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = my_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
run_definition = parameter_values.get("runDefinition")

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Arguments for best run:', run_definition.get("arguments"))
print()
print(best_run.get_file_names())
print()
# Register model
model = best_run.register_model(model_name="my_best_run", model_path="outputs/model.joblib")

# Check model
from azureml.core import Model
for model in Model.list(ws):
    print("Model Name: {}\n".format(model.name))
    print(model)


Best Run Id:  HD_0bca8978-eddd-4fce-8d8a-fff6fca29fb7_2
Accuracy: 0.9109007040543822
Arguments for best run: ['--C', '1.3552902293173101', '--max_iter', '50']

['azureml-logs/55_azureml-execution-tvmps_328749abf804a9cf3a97acde4bcf3a23c6f9360daeb8b57c8d4531bb8db7b053_d.txt', 'azureml-logs/65_job_prep-tvmps_328749abf804a9cf3a97acde4bcf3a23c6f9360daeb8b57c8d4531bb8db7b053_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_328749abf804a9cf3a97acde4bcf3a23c6f9360daeb8b57c8d4531bb8db7b053_d.txt', 'logs/azureml/99_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_b0d12f3f-0e98-4ab6-8ff7-7fe3c393c810.jsonl', 'logs/azureml/dataprep/python_span_l_b0d12f3f-0e98-4ab6-8ff7-7fe3c393c810.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']

Model Name: my_best_run

Model(workspace=Workspace.create(name='quick-starts-ws

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [16]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###